# Notebook Dedicated to Compute the Event Selection Efficiency of MC Data of Solar Nue and 208Tl

In [2]:
import uproot
import numpy as np
import pandas as pd
import random

import glob
import re

import seaborn as sn
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

# Load Data
Use the Train Dataset to Estimate the selection efficiency of solar Nue and 208Tl events

In [13]:
# ========== Define the event selection cuts to compute the efficiency ==========
imag_dim = (10,10)

energy_inf_cut = 2.5
energy_sup_cut = 12

posr_cut = 5500

time_res_inf_cut = -1
time_res_sup_cut = 5

nhits_cut = 20 #Nhits cut to be applied after the time residual cuts

energy_sup_model_train = 7  #Define the energy sup cut used to train the model

# ========== files directory==========
main_dir = f'E:/Data/solars/mc/ML Data/Train/np_arrays/E_{energy_inf_cut}_{energy_sup_cut}_MeV_R_{posr_cut}_mm_Delta_{time_res_inf_cut}_{time_res_sup_cut}_ns/'
solar_fname = 'solarNue_n_init_evs_*.npy'
tl208_fname = '208Tl_n_init_evs_*.npy'

flist_solar = glob.glob(main_dir + solar_fname)
flist_208tl = glob.glob(main_dir + tl208_fname)

## Initial Nº of events

In [5]:
N_init_solar = []  #Nº of initial solar 8B events 
N_init_208tl = []  #Nº of initial 208Tl events

#Loop over solars:
for fsolar_i in flist_solar:
    n_init_ev = np.load(fsolar_i)[0]
    N_init_solar.append(n_init_ev)
N_init_solar = np.array(N_init_solar)
N_init_solar = np.sum(N_init_solar)

#Loop over 208Tl:
for f208tl_i in flist_208tl:
    n_init_ev = np.load(f208tl_i)[0]
    N_init_208tl.append(n_init_ev)

N_init_208tl = np.array(N_init_208tl)
N_init_208tl = np.sum(N_init_208tl)

print(f'N of initial MC 8B Nue events = {N_init_solar}')
print(f'N of initial MC 208Tl events = {N_init_208tl}')

N of initial MC 8B Nue events = 2794531
N of initial MC 208Tl events = 938314


## Final Nº of events: Load the Saved Pandas DataFrame

In [14]:
# ====================== Load the pd.DataFrame ======================
train_dataset_dir = f'E:/Data/solars/mc/ML Data/Train/pd_dataframe/'
fname = f'df_Dim_{imag_dim[0]}_E_{energy_inf_cut}_{energy_sup_cut}_MeV_R_{posr_cut}_mm_Delta_{time_res_inf_cut}_{time_res_sup_cut}_ns.csv'

df_train = pd.read_csv(train_dataset_dir + fname)

# Separate solar 8B Nue from 208Tl Datasets
dataset_sig_train = (df_train[df_train['IsNu'] == 1]).reset_index()
dataset_bkg_train = (df_train[df_train['IsTl'] == 1]).reset_index()

# ========== Apply cuts on pd Dataframe as used to train the ML Model ==========
#cut in solar energy:
solar_energy_condition = (dataset_sig_train['energy (MeV)'] >= energy_inf_cut) & (dataset_sig_train['energy (MeV)'] <= energy_sup_model_train)
dataset_sig_train = (dataset_sig_train[solar_energy_condition])

#cut in 208tl energy:
tl208_energy_condition = (dataset_bkg_train['energy (MeV)'] >= energy_inf_cut) & (dataset_bkg_train['energy (MeV)'] <= energy_sup_model_train)
dataset_bkg_train = (dataset_bkg_train[tl208_energy_condition])
# ===============================================================================

#Extract the Number of final events:
N_final_solar = dataset_sig_train.shape[0]
N_final_208tl = dataset_bkg_train.shape[0]

print(f'N of final MC 8B Nue events = {N_final_solar}')
print(f'N of final MC 208Tl events = {N_final_208tl}')

N of final MC 8B Nue events = 803545
N of final MC 208Tl events = 515054


# Efficiency Computation

In [15]:
solar_eff = N_final_solar/ N_init_solar
tl208_eff = N_final_208tl / N_init_208tl

print(f'MC solar 8B-Nue events selection efficiency = {solar_eff} ')
print(f'MC 208-Thallium events selection efficiency = {tl208_eff} ')

MC solar 8B-Nue events selection efficiency = 0.28754198826207333 
MC 208-Thallium events selection efficiency = 0.5489143293183305 
